In [1]:
!pip install fasttext
!pip install uralicNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 655.5 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.5-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.5-py3-none-any.whl (240 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246560 sha256=e819fe3fb008a4875f05516179739731f083cff1764e3a025d2c287f921add5c
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.8/281.8 kB 4.7 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import fasttext as ft
from uralicNLP import uralicApi

In [2]:
uralicApi.download("mns")

100%|█████████▉| 3161/3161.1767578125 [00:00<00:00, 38629.75it/s]


Model analyser for mns was downloaded


100%|█████████▉| 3906/3906.982421875 [00:00<00:00, 39932.32it/s]


Model morpher-gt-desc.hfstol for mns was downloaded


100%|█████████▉| 34420/34420.298828125 [00:00<00:00, 63132.31it/s]


...

Model cg for mns was downloaded


 86%|████████▌ | 1/1.1640625 [00:00<00:00, 3366.22it/s]


Model metadata.json for mns was downloaded


5737it [00:00, 66989.01it/s]                      

Model dictionary.json for mns was downloaded


In [4]:
df = pd.read_csv("/content/train.csv")["target"]

In [5]:
def replace_symbols(text, f=True):
    for symbol in "!@#$%,.1234567890/?'\"\\«»_:;_…()”":
        text = text.replace(symbol, " ")
    first_words = set((" ".join(text.lower().split())).split())

    for symbol in "!@#$%,.1234567890-/?'\"\\«»_-:;_…–()”":
        text = text.replace(symbol, " ")
    second_words = set((" ".join(text.lower().split())).split())
    return list(first_words | second_words)


def lemmatize_mans_sentence(sentence):
    sentence = replace_symbols(sentence)
    words = []
    for word in sentence:
        norm_form = uralicApi.lemmatize(word, "mns")
        if len(norm_form) == 0:
            words.append(word)
        else:
            words.extend(sorted(uralicApi.lemmatize(word, "mns")))
    return list(set(words) | set(sentence))


In [6]:
df = df.apply(lemmatize_mans_sentence)

In [7]:
df = df.apply(" ".join)

In [8]:
np.savetxt(r'/content/train.txt', df.values, "%s")

In [9]:
model = ft.train_unsupervised('/content/train.txt')

In [ ]:
model.get_nearest_neighbors("школат")

[(0.9603418707847595, 'школа'),
 (0.948723554611206, 'школатт'),
 (0.9435409307479858, 'школавт'),
 (0.9430932402610779, 'школав'),
 (0.9395305514335632, 'ишколат'),
 (0.9335001111030579, 'школатыт'),
 (0.9147893786430359, 'школант'),
 (0.903683602809906, 'исколат'),
 (0.8910618424415588, 'школатын'),
 (0.8715890645980835, 'школа-интернатт')]

In [25]:
def get_sentence_embeddings(model, df):
    embeddings = []
    for sentence in df:
        sentence = sentence.split()
        sentence_embedding = model.get_word_vector(sentence[0])
        for word in sentence[1:]:
            sentence_embedding = np.add(model.get_word_vector(word), sentence_embedding)
        sentence_embedding = sentence_embedding / len(sentence)
    embeddings.append(sentence_embedding)
    return embeddings